## Optimization Program for Venue Series
### Copyright © 2024 Hoshino Math Services

In [1]:
# Import the Python Modules.

import time
import math
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp


# Import the Input File

InputFile = pd.ExcelFile("Venue Series Input File.xlsx") 

VendorMatrix = pd.read_excel(InputFile, "Vendors") 
VendorInfo = VendorMatrix.values.tolist()
AttendeeMatrix = pd.read_excel(InputFile, "Attendees") 
AttendeeInfo = AttendeeMatrix.values.tolist()

VCMatrix = pd.read_excel(InputFile, "Vendor Choices") 
ACMatrix = pd.read_excel(InputFile, "Attendee Choices") 

# Generate the list of Vendors and Attendees that are present

VendorList = []
for i in range(len(VendorInfo)):
    if pd.notna(VendorInfo[i][3]):
        VendorList.append(VendorInfo[i][0])
VendorList.sort()
    
AttendeeList = []
for i in range(len(AttendeeInfo)):
    if pd.notna(AttendeeInfo[i][3]):
        AttendeeList.append(AttendeeInfo[i][0])
AttendeeList.sort()

V = len(VendorList)
A = len(AttendeeList)

print("There are", V, "vendors and", A, "attendees that are present")

There are 20 vendors and 30 attendees that are present


In [2]:
# Determine the score of Vendor V being matched with Attendee A

VAScore = [[0 for a in range(A)] for v in range(V)]


# Assign 3 points for each occurrence of an Attendee picking a Vendor

for VendorID in VendorList:
    temp = ACMatrix[VendorID]
    for i in range(len(temp)):
        if ACMatrix["Attendee ID"][i] in AttendeeList:
            VAScore[VendorList.index(VendorID)][AttendeeList.index(ACMatrix["Attendee ID"][i])] += 3*temp[i]
            
            
# Assign 2 points for each occurrence of a Vendor picking an Attendee

for AttendeeID in AttendeeList:
    temp = VCMatrix[AttendeeID]
    for i in range(len(temp)):
        if VCMatrix["Vendor ID"][i] in VendorList:
            VAScore[VendorList.index(VCMatrix["Vendor ID"][i])][AttendeeList.index(AttendeeID)] += 2*temp[i]
            
            
# Assign -100 points for each occurrence of (Vendor, Attendee) pair where neither side has picked the other

for v in range(V):
    for a in range(A):
        if VAScore[v][a] == 0:
            VAScore[v][a] = -100
            
        
# Determine the number of meeting rounds

ParameterMatrix = pd.read_excel(InputFile, "Parameters") 
ParameterInfo = ParameterMatrix.values.tolist()
R = ParameterInfo[0][2]

In [3]:
# Optimize the assignment of Attendees to Vendors

StartTime = time.time()

solver = pywraplp.Solver("Venue Series", pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

Vendors = range(len(VendorList))
Attendees = range(len(AttendeeList))
Rounds = range(ParameterInfo[0][2])

x = {}
for v in Vendors:
    for a in Attendees:
        for r in Rounds:
            x[v,a,r] = solver.IntVar(0,1, "x[%d,%d,%d]" % (v,a,r))

        
# CONSTRAINT 1: Each Vendor has one Attendee Meeting per round
for v in Vendors:
    for r in Rounds:
        solver.Add(sum(x[v,a,r] for a in Attendees) == 1)
        
        
# CONSTRAINT 2: Each Attendee has at most one Vendor Meeting per round
for a in Attendees:
    for r in Rounds:
        solver.Add(sum(x[v,a,r] for v in Vendors) <= 1)
        
        
# CONSTRAINT 3: Each Attendee has approximately the same number of Vendor Meetings
for a in Attendees:
    solver.Add(sum(x[v,a,r] for v in Vendors for r in Rounds) >= math.floor(V*R/A))
    solver.Add(sum(x[v,a,r] for v in Vendors for r in Rounds) <= math.ceil(V*R/A))
    
    
# CONSTRAINT 4: Each (Vendor, Attendee) pair can only meet once
for v in Vendors:
    for a in Attendees:
        solver.Add(sum(x[v,a,r] for r in Rounds) <= 1)
    
        
# CONSTRAINT 5: If the Vendor is not Local and the Attendee is not National, they cannot be paired

NonLocalVendorList = []
for i in range(len(VendorInfo)):
    if pd.isna(VendorInfo[i][4]): NonLocalVendorList.append(VendorInfo[i][0])
    
NonNationalAttendeeList = []
for i in range(len(AttendeeInfo)):
    if pd.isna(AttendeeInfo[i][4]): NonNationalAttendeeList.append(AttendeeInfo[i][0])

for v in Vendors:
    for a in Attendees:
        if VendorList[v] in NonLocalVendorList and AttendeeList[a] in NonNationalAttendeeList:
            solver.Add(sum(x[v,a,r] for r in Rounds) == 0)
            
    
# Run the optimization
solver.Maximize(solver.Sum(VAScore[v][a] * x[v,a,r] for v in Vendors for a in Attendees for r in Rounds))
sol = solver.Solve()

Solution = []
for v in Vendors:
    for a in Attendees:
        for r in Rounds:
            if x[v,a,r].solution_value()==1:
                Solution.append([v,a,r])
                
TotalScore = 0
for temp in Solution:
    TotalScore += VAScore[temp[0]][temp[1]]
      
TotalTime = time.time() - StartTime
print("Optimization Complete in", round(TotalTime,4), "seconds")   
print("The total score is", TotalScore, "points")

Optimization Complete in 9.9194 seconds
The total score is 260 points


In [4]:
for temp in Solution:
    if VAScore[temp[0]][temp[1]] < 1:
        print(VendorList[temp[0]], "and", AttendeeList[temp[1]], "has a score of", VAScore[temp[0]][temp[1]], "points")

V07 and A08 has a score of -100 points
V07 and A12 has a score of -100 points
V07 and A16 has a score of -100 points
V07 and A24 has a score of -100 points
V07 and A32 has a score of -100 points
V07 and A36 has a score of -100 points
V07 and A40 has a score of -100 points
V07 and A48 has a score of -100 points
V07 and A56 has a score of -100 points
V07 and A64 has a score of -100 points
V07 and A72 has a score of -100 points
V07 and A80 has a score of -100 points
V13 and A24 has a score of -100 points


In [5]:
# Export to Excel

OurColumns = ["Attendee ID", "Attendee Name", "Attendee Company", "Present", "National"]
for r in Rounds:
    OurColumns.append("Round " + str(r+1))

M = []
for i in range(len(AttendeeInfo)):
    AllVendors = ["" for r in Rounds]
    if AttendeeInfo[i][0] in AttendeeList:
        AttendeeID = AttendeeList.index(AttendeeInfo[i][0])
        for mypair in Solution:
            if mypair[1] == AttendeeID:
                AllVendors[mypair[2]] = VendorList[mypair[0]]
    M += [AttendeeInfo[i][0:5] + AllVendors[0:R]]
FinalMatrix = pd.DataFrame(M, columns=OurColumns)

with pd.ExcelWriter('Output File - Attendee Assignments.xlsx') as writer:
    FinalMatrix.to_excel(writer, sheet_name="Attendees", index=False)
    

OurColumns = ["Vendor ID", "Vendor Name", "Vendor Hotel", "Present", "Local"]
for r in Rounds:
    OurColumns.append("Round " + str(r+1))

M = []
for i in range(len(VendorInfo)):
    AllAttendees = ["" for r in Rounds]
    if VendorInfo[i][0] in VendorList:
        VendorID = VendorList.index(VendorInfo[i][0])
        for mypair in Solution:
            if mypair[0] == VendorID:
                AllAttendees[mypair[2]] = AttendeeList[mypair[1]]
    M += [VendorInfo[i][0:5] + AllAttendees[0:R]]
FinalMatrix = pd.DataFrame(M, columns=OurColumns)

with pd.ExcelWriter('Output File - Vendor Assignments.xlsx') as writer:
    FinalMatrix.to_excel(writer, sheet_name="Vendors", index=False)
    